<a href="https://colab.research.google.com/github/Saravanan-039/ML-challenge/blob/main/T22103.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

d= pd.read_csv('labelled_train_set.csv')

X = d.drop(['Type', 'ID'], axis=1)
y = d['Type']

vect = TfidfVectorizer()
X = vect.fit_transform(X['Article'])

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=100)

model = SGDClassifier(loss='hinge', random_state=42)

model.fit(Xtrain, ytrain)

ypred = model.predict(Xtest)

df = pd.read_csv('unlabelled_test2.csv')
pt = df['Article']

pv = vect.transform(pt)
ypred = model.predict(pv)

for test_id, prediction in zip(df['ID'], ypred):
    predicted_label = 0 if prediction == "Human-written" else 1

labels = [0 if prediction == "Human-written" else 1 for prediction in ypred]

results = pd.DataFrame({
    'ID': df['ID'],
    'Predicted Type': labels
})
results.to_csv('predictions.csv', index=False)

In [ ]:
#2

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

d = pd.read_csv('labelled_train_set.csv')

X = d.drop(['Type', 'ID'], axis=1)
y = d['Type']

vect = TfidfVectorizer()
X = vect.fit_transform(X['Article'])

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

model = Perceptron()

model.fit(Xtrain, ytrain)

ypred = model.predict(Xtest)

df = pd.read_csv('unlabelled_test2.csv')
text = df['Article']

vectors = vect.transform(text)
ypred = model.predict(vectors)

labels = [0 if prediction == "Human-written" else 1 for prediction in ypred]

result = pd.DataFrame({
    'ID': df['ID'],
    'Predicted Type': labels
})


result.to_csv('prediction2.csv', index=False)



In [ ]:
!pip install emoji


import pandas as pd
import re
import emoji
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

def preprocess_text(text):
    text = emoji.replace_emoji(text, replace=' ')
    text = re.sub(r'[^\w\s]', '', text)
    return text.lower()

d = pd.read_csv('labelled_train_set.csv')

d['Article'] = d['Article'].apply(preprocess_text)

X = d.drop(['Type', 'ID'], axis=1)
y = d['Type']

vect = TfidfVectorizer(ngram_range=(1, 3))
X = vect.fit_transform(X['Article'])

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=100)

grid = {
    'loss': ['hinge', 'log_loss', 'modified_huber'],
    'alpha': [1e-4, 1e-3, 1e-2],
    'max_iter': [1000, 2000],
    'class_weight': [None, 'balanced']
}

grid_search = GridSearchCV(SGDClassifier(random_state=42), grid, cv=5, scoring='f1_weighted')
grid_search.fit(Xtrain, ytrain)

model = grid_search.best_estimator_

model.fit(Xtrain, ytrain)

pred = model.predict(Xtest)

df = pd.read_csv('unlabelled_test2.csv')
ptext = df['Article']

vectors = vect.transform(ptext)
pred = model.predict(vectors)

for test_id, prediction in zip(df['ID'], pred):
    predicted_label = 0 if prediction == "Human-written" else 1

label = [0 if prediction == "Human-written" else 1 for prediction in pred]

result = pd.DataFrame({
    'ID': df['ID'],
    'Predicted Type': label
})

result.to_csv('predictions3.csv', index=False)